In [1]:
from lark import Lark

In [2]:
aut_parser = Lark.open("grammars/aut.lark", parser="earley")
mu_parser = Lark.open("grammars/mu.lark", parser="earley")

In [3]:
system = aut_parser.parse(open("../examples/test.aut").read())
formula = mu_parser.parse(open("../examples/test.mu").read())
print(system.pretty())
print(formula.pretty())

start
  aut_header
    0
    12
    10
  aut_edge
    0
    "lock(p2, f2)"
    1
  aut_edge
    0
    "lock(p1, f1)"
    2
  aut_edge
    1
    "lock(p1, f1)"
    3
  aut_edge
    1
    "lock(p2, f1)"
    4
  aut_edge
    2
    "lock(p2, f2)"
    3
  aut_edge
    2
    "lock(p1, f2)"
    5
  aut_edge
    4
    "eat(p2)"
    6
  aut_edge
    5
    "eat(p1)"
    7
  aut_edge
    6
    "free(p2, f2)"
    8
  aut_edge
    7
    "free(p1, f1)"
    9
  aut_edge
    8
    "free(p2, f1)"
    0
  aut_edge
    9
    "free(p1, f2)"
    0

start
  formula
    or
      formula
        diamond
          a
          formula	true
      formula
        box
          b
          formula
            and
              formula	false
              formula	X



In [4]:
class LabeledTransitionSystem:
    def __init__(self, states, ):


SyntaxError: incomplete input (3476080657.py, line 2)